In [1]:
import numpy as np
import numpy
from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct
from numpy import array, linspace
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import plot
from scipy.signal import argrelextrema
from scipy.ndimage import gaussian_filter1d
import random
import os

In [2]:
'''Road map
----[requirements]
0. install all requirements
    -- pip install -r requirements.txt

----[prepare training data]
1. turn training data into (-1,512,4)  
    : 512 = input_size, 
    : 4= size of the CP token (Bar (0=new,1=continue,2=pad),Position,Pitch,Duration)
    ** DO NOT concatenate all songs tgt, pad every song and make it divisible by 512
    ** pad it with (2,16,86,64)
    e.g. [ [song1-0:511],[song1-512:520+pad],[song2-0:511],[song2-512-530+pad]] ==> shape(4,512,4)
    
2. Prepare answer dataset 
    : (0 = padding, 1 = keep, 2 = discard)
* save it in .npy format
* split into 3 different groups
    put inside ~/data/CP/
    -custom_reduction_train.npy , custom_reduction_train_ans.npy
    -custom_reduction_valid.npy , custom_reduction_valid_ans.npy
    -custom_reduction_test.npy  , custom_reduction_test_ans.npy
    
** using  .py
    - cd to ./prepare_data/CP
    - python main.py --task reduction --input_dir "../../dataset(orchestra only)"
    >files will be saved at ~/data/CP/
    >please move all the files to the correct directory


    
----[start fine tuning]
1. cd to ./MidiBERT/CP
2. python3 finetune.py --task=reduction  --epochs 1 --ckpt xxxxx (default='result/finetune/pretrain_model.ckpt')
* gpu with --cuda_devices 0 (+1)
* if gpu is available and --cpu flag is not set, by default it will be trained on GPU

----[eval](not finished yet)
1. cd to ./MidiBERT/CP
2. python3 eval.py --task=reduction

'''

"Road map\n----[requirements]\n0. install requirement\n    -- pip install -r requirements.txt\n\n----[prepare training data]\n1. turn training data into (-1,512,4)  \n    : 512 = input_size, \n    : 4= size of CP token (includingBar (0=new,1=continue,2=pad),Position,Pitch,Duration)\n    ** DO NOT concatenate all songs tgt, pad every song and make it divisible by 512\n    ** pad it with (2,16,86,64)\n    e.g. [ [song1-0:511],[song1-512:520+pad],[song2-0:511],[song2-512-530+pad].... ]\n    \n2. Prepare answer dataset \n    : (0 = padding, 1 = keep, 2 = discard)\n* save in .npy format\n* split in 3 different groups\n    dataroot= ~/data/CP/\n    -custom_reduction_train.npy , custom_reduction_train_ans.npy\n    -custom_reduction_valid.npy , custom_reduction_valid_ans.npy\n    -custom_reduction_test.npy  , custom_reduction_test_ans.npy\n\n    \n----[start fine tuning]\n1. cd to ./MidiBERT/CP\n2. python3 finetune.py --task=reduction  --epochs 1 --ckpt xxxxx (default='result/finetune/pretrain

In [3]:
#DEMO

In [4]:
#training data after tokenization
a=np.load('./data/CP/custom.npy') # after tokenization
a.shape

(4, 512, 4)

In [5]:
#corrsponding ans
fake_ans= np.array([[random.randint(1,2) for _ in range(512)] for __ in range(4)])  #random generate
fake_ans[3][206:]=0 # random padding
fake_ans=fake_ans.astype(np.int64)

In [6]:
#test,train,valid split
#assume a[0],a[1],a[2:] are three different songs
testX,testY = a[0].reshape(-1,512,4),fake_ans[0].reshape(-1,512)
validX,validY = a[1].reshape(-1,512,4),fake_ans[1].reshape(-1,512)
trainX,trainY = a[2:].reshape(-1,512,4),fake_ans[2:].reshape(-1,512)

In [7]:
#save training data
np.save('./data/CP/custom_reduction_train.npy',trainX)
np.save('./data/CP/custom_reduction_train_ans.npy',trainY)
np.save('./data/CP/custom_reduction_valid.npy',validX)
np.save('./data/CP/custom_reduction_valid_ans.npy',validY)
np.save('./data/CP/custom_reduction_test.npy',testX)
np.save('./data/CP/custom_reduction_test_ans.npy',testY)

In [8]:
# start fine tuning
! python ./MidiBERT/CP/finetune.py --task=reduction --epochs 1

C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
Loading Dictionary

Loading Dataset
X_train: (2, 512, 4), X_valid: (1, 512, 4), X_test: (1, 512, 4)
y_train: (2, 512), y_valid: (1, 512), y_test: (1, 512)
   len of train_loader 1
   len of valid_loader 1
   len of valid_loader 1

Building BERT model
   Loading pre-trained model from pretrain_model.ckpt

D:\anacnda\envs\RL\lib\site-packages\torch\utils\data\dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))

100%|██████████| 1/1 [00:42<00:00, 42.76s/it]

100%|██████████| 1/1 [00:26<00:00, 26.03s/it]

100%|██████████| 1/1 [00:22<00:00, 22.56s/it]




Creating Finetune Trainer using index layer -1
   device: cpu
init a fine-tune model, sequence-level task? False

Training Start
   save model at C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/result/finetune/reduction_\model.ckpt
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP/
1
513.0
epoch: 1/1 | Train Loss: 1.0158 | Train

In [ ]:
# eval
'''
procedure:
1. use prepare_data to tokenlize *ONE* song to .npy format
    -- it should be saved under ~/data/CP
2. cd to ~/MidiBERT/CP
3. python eval.py --task reduction --case [file name of the .npy file]
3. two mid are generated 


--[sample]
1. put orchestra.mid into testcase folder *(the file should only contain 1 file)*
2. cd ./prepare_data/CP
3. python .\main.py --task custom --name testcase1 --input_dir ../../testcase
4. cd ../../MidiBERT/CP
5. python eval.py --task reduction --case testcase1
6. u can find the two files in ~/MidiBERT/CP now
'''

In [5]:
cd ./prepare_data/CP
python main.py --task custom --name testcase1 --input_dir ../../testcase
cd ../../CP
python eval.py --task reduction --case testcase1
cd ../..


python: can't open file 'main.py': [Errno 2] No such file or directory


# case study

In [2]:
'''
please reload the notebook if you hv messed up the working path
How to use: run below
'''

'\nplease reload the notebook if you hv messed up the working path\nHow to use: run below\n'

In [3]:
import numpy as np
import numpy
from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct
from numpy import array, linspace
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import plot
from scipy.signal import argrelextrema
from scipy.ndimage import gaussian_filter1d
import random
import os
def get_length(m):
    mido_obj = mid_parser.MidiFile(m)
    notes = [] 
    for instrument in mido_obj.instruments:
        for note in instrument.notes:
            notes.append(note)
    return len(notes)
def read_midi(path):
    mido_obj = mid_parser.MidiFile(path)
    tick_per_beat = mido_obj.ticks_per_beat

    notes = [] 
    for instrument in mido_obj.instruments:
        for note in instrument.notes:
            notes.append(note)

    # sort by start time
    notes.sort(key=lambda note:note.start)
    return notes,tick_per_beat

def write_midi(notes,tick_per_beat=480,path='out.mid'):
    out = mid_parser.MidiFile()
    out.ticks_per_beat = tick_per_beat
    out.instruments = [ct.Instrument(program=0,is_drum=False,name='post-processed piano')]
    for note in notes:
        assert(note.velocity)
        out.instruments[0].notes.append(ct.Note(start=note.start,end=note.end,pitch=note.pitch,velocity=note.velocity))
    out.dump(path)

In [5]:
a,b=read_midi('token2mid.mid')

In [3]:
import os
base_dir = os.path.dirname(os.path.realpath('__file__'))
print(base_dir)

C:\Users\tokah\Documents\MIDI-BERT


In [4]:
%pwd

'C:\\Users\\tokah\\Documents\\MIDI-BERT'

In [1]:
# file inside casestudySamples folder
file='InTheHallOfTheMountainKing.mid'

In [6]:
%cd {base_dir}

C:\Users\tokah\Documents\MIDI-BERT


In [7]:
#move the testcase from casestudysample to the desired folder
%cd ./testcase
!del /q *
!copy ..\casestudySamples\{file} "./"
!copy ..\casestudySamples\{file} "../"
%cd ../

#prepare .npy data
%cd ./prepare_data/CP
!python .\main.py --task custom --name testcase1 --input_dir ../../testcase

#run reduction
%cd ../../MidiBERT/CP
!python eval.py --task reduction --case testcase1
!move ./reduction.mid ../../reduction.mid

#run postprocessing
%cd ../..
!python postprocessing.py -f reduction.mid
%cd {base_dir}
a,b=read_midi('./testcase/'+file)
write_midi(a,b,'merged_orchestra.mid')

#token2mid
c=np.load('./data/CP/testcase1.npy')
out = mid_parser.MidiFile()
out.ticks_per_beat = 1024
out.instruments = [ct.Instrument(program=0,is_drum=False,name='reduction')]
current_beat=-1
for idx1,i in enumerate(c):
    for idx2,j in enumerate(i):                    
        n=c[idx1][idx2]
        if n[0]==0:
            current_beat+=1
        if c[idx1][idx2][0]!=2:
            out.instruments[0].notes.append(ct.Note(start=int(current_beat*4*out.ticks_per_beat+n[1]*out.ticks_per_beat/4),
                                                    end=int(current_beat*4*out.ticks_per_beat+n[1]*out.ticks_per_beat/4+(n[3]+1)*(out.ticks_per_beat/8)),
                                                    pitch=n[2]+22,
                                                    velocity=90))
out.dump('token2mid.mid')


#group files tgt
!rmdir /q {file[:-4]}
%mkdir {file[:-4]}
!echo original_size:{get_length(file)}  -  reduced_size:{get_length('reduction.mid')}  -  postprocessed_size:{get_length('post_processed.mid')} > ./{file[:-4]}/log.txt
!move ./reduction.mid ./{file[:-4]}/
!move ./token2mid.mid ./{file[:-4]}/
!move ./{file} ./{file[:-4]}/
!move ./merged_orchestra.mid ./{file[:-4]}/
!move ./post_processed.mid ./{file[:-4]}/
!echo finished

C:\Users\tokah\Documents\MIDI-BERT\testcase
複製了         1 個檔案。
複製了         1 個檔案。
C:\Users\tokah\Documents\MIDI-BERT
C:\Users\tokah\Documents\MIDI-BERT\prepare_data\CP
Number of  files: 1
Data shape: (12, 512, 4), saved at ../../data/CP\testcase1.npy
C:\Users\tokah\Documents\MIDI-BERT\MidiBERT\CP


2022-03-22 20:41:07,015 - model - INFO - ../../testcase\InTheHallOfTheMountainKing.mid


Loading Dictionary

Building BERT model

Loading Dataset
   len of predict_loader 1

Load ckpt from result/finetune/reduction_/model_best.ckpt

Creating Finetune Trainer using index layer -1
! (12, 512, 4) (12, 512)
   device: cpu
load a fine-tuned model
1
1.0
reduced 2329 notes


D:\anacnda\envs\RL\lib\site-packages\torch\utils\data\dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


移動         1 個檔案。
C:\Users\tokah\Documents\MIDI-BERT
before post-processing 3805 notes
Octave Transpose: #notes 3805 transposed 386 notes , skipped 620
Merge discrete: merged  370  discrete notes
Drop discrete: dropped  0  discrete notes
Doubling Simplification: removed doubling 621
after post-processing, total notes: 2814, removed 991 notes in total
Figure(2000x2000)
before post-processing 2814 notes
Octave Transpose: #notes 2814 transposed 191 notes , skipped 658
Merge discrete: merged  15  discrete notes
Drop discrete: dropped  0  discrete notes
Doubling Simplification: removed doubling 1
after post-processing, total notes: 2798, removed 16 notes in total
Figure(2000x2000)
before post-processing 2798 notes
Octave Transpose: #notes 2798 transposed 64 notes , skipped 634
Merge discrete: merged  2  discrete notes
Drop discrete: dropped  0  discrete notes
Doubling Simplification: removed doubling 0
after post-processing, total notes: 2796, removed 2 notes in total
Figure(2000x2000)
C:\U

目錄不是空的。
子目錄或檔案 InTheHallOfTheMountainKing 已經存在。


移動         1 個檔案。
移動         1 個檔案。
移動         1 個檔案。
移動         1 個檔案。
移動         1 個檔案。
finished


In [ ]:
#linux version
%cd ./testcase
# linus: %rm ./*
!rm -f ./*
!cp ../casestudySamples/{file} "./"
!cp ../casestudySamples/{file} "../"
%cd ../
%cd ./prepare_data/CP
!python .\main.py --task custom --name testcase1 --input_dir ../../testcase
%cd ../../MidiBERT/CP
!python eval.py --task reduction --case testcase1
#linux#%mv reduction.mid ../../reduction.mid
!mv ./reduction.mid ../../reduction.mid
%cd ../..
!python postprocessing.py -f reduction.mid
%cd base_dir
a,b=read_midi('./testcase/'+file)
write_midi(a,b,'merged_orchestra.mid')

c=np.load('./data/CP/testcase1.npy')
out = mid_parser.MidiFile()
out.ticks_per_beat = 1024
out.instruments = [ct.Instrument(program=0,is_drum=False,name='reduction')]
current_beat=-1
for idx1,i in enumerate(c):
    for idx2,j in enumerate(i):                    
        n=c[idx1][idx2]
        if n[0]==0:
            current_beat+=1


        if c[idx1][idx2][0]!=2:
            out.instruments[0].notes.append(ct.Note(start=int(current_beat*4*out.ticks_per_beat+n[1]*out.ticks_per_beat/4),
                                                    end=int(current_beat*4*out.ticks_per_beat+n[1]*out.ticks_per_beat/4+(n[3]+1)*out.ticks_per_beat/8),
                                                    pitch=n[2]+22,
                                                    velocity=30))
out.dump('token2mid.mid')


!rm -r -f {file[:-4]}
%mkdir {file[:-4]}
!echo original_size:{get_length(file)}\nreduced_size:{get_length('reduction.mid')}\npostprocessed_size:{get_length('post_processed.mid')} > ./{file[:-4]}/log.txt
!mv ./reduction.mid ./{file[:-4]}/
!mv ./token2mid.mid ./{file[:-4]}/
!mv ./{file} ./{file[:-4]}/
!mv ./merged_orchestra.mid ./{file[:-4]}/
!mv ./post_processed.mid ./{file[:-4]}/
!echo finished

# DEBUG 

In [22]:
a,b=read_midi('./token2mid.mid')


In [ ]:
write_midi(a,b,'merged_orchestra.mid')

In [8]:
c=np.load('./data/CP/testcase1.npy')

In [23]:
b

1024

In [20]:
len(a),len(c)

(5957, 12)

In [10]:
c[0][:50]

array([[ 0,  0, 32, 32],
       [ 1,  0, 44, 32],
       [ 0,  0, 25,  0],
       [ 1,  0, 13,  0],
       [ 1,  0, 25,  2],
       [ 1,  0, 13,  2],
       [ 1,  2, 27,  2],
       [ 1,  2, 15,  2],
       [ 1,  4, 32,  0],
       [ 1,  4, 20,  0],
       [ 1,  4, 28,  2],
       [ 1,  4, 16,  2],
       [ 1,  6, 30,  2],
       [ 1,  6, 18,  2],
       [ 1,  8, 25,  0],
       [ 1,  8, 13,  0],
       [ 1,  8, 32,  2],
       [ 1,  8, 20,  2],
       [ 1, 10, 28,  2],
       [ 1, 10, 16,  2],
       [ 1, 12, 32,  0],
       [ 1, 12, 20,  0],
       [ 1, 12, 32,  6],
       [ 1, 12, 20,  6],
       [ 0,  0, 25,  0],
       [ 1,  0, 13,  0],
       [ 1,  0, 31,  2],
       [ 1,  0, 19,  2],
       [ 1,  2, 27,  2],
       [ 1,  2, 15,  2],
       [ 1,  4, 32,  0],
       [ 1,  4, 20,  0],
       [ 1,  4, 31,  6],
       [ 1,  4, 19,  6],
       [ 1,  8, 25,  0],
       [ 1,  8, 13,  0],
       [ 1,  8, 30,  2],
       [ 1,  8, 18,  2],
       [ 1, 10, 26,  2],
       [ 1, 10, 14,  2],


In [30]:
out = mid_parser.MidiFile()
out.ticks_per_beat = 1024
out.instruments = [ct.Instrument(program=0,is_drum=False,name='reduction')]
current_beat=-1
for idx1,i in enumerate(c):
    for idx2,j in enumerate(i):                    
        n=c[idx1][idx2]
        if n[0]==0:
            current_beat+=1


        if c[idx1][idx2][0]!=2:
            out.instruments[0].notes.append(ct.Note(start=int(current_beat*4*out.ticks_per_beat+n[1]*out.ticks_per_beat/4),
                                                    end=int(current_beat*4*out.ticks_per_beat+n[1]*out.ticks_per_beat/4+n[3]*out.ticks_per_beat/8),
                                                    pitch=n[2]+22,
                                                    velocity=30))
out.dump('token2mid.mid')

In [196]:
c[0][:10]

array([[ 0,  0, 60, 14],
       [ 1,  0, 66, 14],
       [ 1,  0, 48, 14],
       [ 1,  0, 54, 14],
       [ 1,  0, 38, 19],
       [ 1,  0, 48,  7],
       [ 1,  0, 45, 14],
       [ 1,  0, 48, 14],
       [ 1,  0, 26, 19],
       [ 1,  0, 54,  7]])

In [197]:
a[:10]

[Note(start=481, end=711, pitch=82, velocity=55),
 Note(start=481, end=711, pitch=88, velocity=55),
 Note(start=482, end=712, pitch=70, velocity=70),
 Note(start=482, end=712, pitch=76, velocity=70),
 Note(start=482, end=782, pitch=60, velocity=82),
 Note(start=482, end=602, pitch=70, velocity=42),
 Note(start=483, end=713, pitch=70, velocity=70),
 Note(start=483, end=713, pitch=67, velocity=70),
 Note(start=484, end=784, pitch=48, velocity=82),
 Note(start=484, end=604, pitch=76, velocity=42)]

In [198]:
out.instruments[0].notes[:10]

[Note(start=0, end=1792, pitch=82, velocity=30),
 Note(start=0, end=1792, pitch=88, velocity=30),
 Note(start=0, end=1792, pitch=70, velocity=30),
 Note(start=0, end=1792, pitch=76, velocity=30),
 Note(start=0, end=2432, pitch=60, velocity=30),
 Note(start=0, end=896, pitch=70, velocity=30),
 Note(start=0, end=1792, pitch=67, velocity=30),
 Note(start=0, end=1792, pitch=70, velocity=30),
 Note(start=0, end=2432, pitch=48, velocity=30),
 Note(start=0, end=896, pitch=76, velocity=30)]